# **RAG model - NLP - házi feladat**
AX83OL

## Rövid ismertető

A következő colbertv2.0 ragatouille előtanított model a PTE valamint a PTE EHÖK hivatalos szabályzatain lett továbbtanítva.

Az alábbi weboldalak alatt lévő dokumentumokat használja:

*   A Pécsi Tudományegyetem Szervezeti és Működési Szabályzata és mellékletei:

    https://adminisztracio.pte.hu/adminisztracio/szabalyzatok_utasitasok/szervezeti_es_mukodesi_szabalyzat_es_mellekletek

*   PTE EHÖK Dokumentumtár:

    https://pteehok.hu/dokumentumtar/

Az összes dokumentum manuálisan le lett töltve a két oldalról, majd feltöltve ZIP fájlként a Google Drive-ra, onnan tud ez a notebook hozzáférni.

A továbbtanításhoz használni kívánt PDF-ek szerencsére szövegként tartalmaznak mindent, így azok felhasználhatóak voltak. (Illetve volt talán 2-3 melléklet ami nem, de azok amúgy sem túl kardinális témáról szólnak, így azokat kihagytam.)

Az összes használt doksi (=tanítóhalmaz) együttvéve kb. 1800 oldalt tesz ki.

# MEGVALÓSÍTÁS:

Feltelepítjük a két alapvetően szükséges modult (CUDA-hoz és RAG-hoz kell).

In [ ]:
!pip install optimum auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.9/409.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 978.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
!pip install ragatouille

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.0

Betöltjük az adott modelt.

In [ ]:
from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

Kell majd a PyPDF2 modul a PDF-ek feldolgozásához, így azt is feltelepítjük.

In [ ]:
!pip install PyPDF2

Készítünk egy függvényt ami a Google Drive-ról letöltött és már kicsomagolt mappában lévő összes PDF doksin végigmegy, majd azokat betölti egy string változóba.

In [ ]:
import os
from PyPDF2 import PdfReader

def read_pdfs(directory):
    """
    Read all PDF files in the specified directory and concatenate their contents into a single string.

    :param directory: str - Path to the directory containing PDF files.
    :return: str - Concatenated text content of all PDF files.
    """
    full_text = ""

    # Iterate through all files in the directory
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)

        # Check if the file is a PDF
        if filepath.endswith(".pdf"):
            with open(filepath, "rb") as file:
                reader = PdfReader(file)

                # Iterate through each page of the PDF
                for page_num in range(len(reader.pages)):
                    page = reader.pages[page_num]
                    full_text += page.extract_text()

    return full_text

Letöltjük a forrásadatokat.

In [ ]:
!gdown 16Pn9bYZSY3no-QXvwKlsK0ORrdaG0TQX

Downloading...
From (original): https://drive.google.com/uc?id=16Pn9bYZSY3no-QXvwKlsK0ORrdaG0TQX
From (redirected): https://drive.google.com/uc?id=16Pn9bYZSY3no-QXvwKlsK0ORrdaG0TQX&confirm=t&uuid=c09b74c8-85b5-45dd-b439-adb0bc449fb5
To: /content/Szabalyzatok.zip
100% 39.3M/39.3M [00:05<00:00, 7.16MB/s]


Kicsomagoljuk a letöltött ZIP-et, hogy alkalmazni tudjuk az így kapott mappára a read_pdfs függvényt.

In [ ]:
import zipfile

# A ZIP fájl elérési útja
zip_file_path = "Szabalyzatok.zip"

# A célkönyvtár, ahova kicsomagoljuk a fájlokat
extract_to_directory = ""

# Megnyitjuk a ZIP fájlt és kicsomagoljuk a tartalmát
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_directory)

print("A ZIP fájl sikeresen kicsomagolva.")

A ZIP fájl sikeresen kicsomagolva.


Alkalmazzuk az előbb említett függvényt.

In [ ]:
# Directory containing the PDF files
pdf_directory = "Szabalyzatok"

# Read all PDF files in the directory and concatenate their contents
full_document = read_pdfs(pdf_directory)

# Check the length of the concatenated text
print(len(full_document))
type(full_document)

5397394


str

In [ ]:
full_document

Output hidden; open in https://colab.research.google.com to view.

### Indexeljük a tanítóhalmazt, itt hossznak 180-at adtam meg: ez azt jelenti, hogy milyen hosszú részekre darabolja fel a tanítóhalmazt. => Majd a válaszok milyen hosszúak legyenek max.

In [74]:
RAG.index(collection=[full_document], index_name="PTE", max_document_length=180, split_documents=True)

INFO: Tried to clean up context but failed!
---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Apr 06, 15:27:55] #> Note: Output directory .ragatouille/colbert/indexes/PTE already exists


[Apr 06, 15:27:55] #> Will delete 10 files already at .ragatouille/colbert/indexes/PTE in 20 seconds...
[Apr 06, 15:28:18] [0] 		 #> Encoding 18197 passages..
[Apr 06, 15:29:25] [0] 		 avg_doclen_est = 121.38655090332031 	 len(local_sample) = 18,197
[Apr 06, 15:29:27] [0] 		 Creating 16,384 partitions.
[Apr 06, 15:29:27] [0] 		 *Estimated* 2,208,871 embeddings.
[Apr 06, 15:29:27] [0] 		 #> Saving the index

0it [00:00, ?it/s]

[Apr 06, 15:29:32] [0] 		 #> Encoding 18197 passages..


1it [01:07, 67.75s/it]
100%|██████████| 1/1 [00:00<00:00, 162.82it/s]

[Apr 06, 15:30:40] #> Optimizing IVF to store map from centroids to list of pids..
[Apr 06, 15:30:40] #> Building the emb2pid mapping..


[Apr 06, 15:30:40] len(emb2pid) = 2208871


100%|██████████| 16384/16384 [00:00<00:00, 71065.93it/s]

[Apr 06, 15:30:40] #> Saved optimized IVF to .ragatouille/colbert/indexes/PTE/ivf.pid.pt


Done indexing!


'.ragatouille/colbert/indexes/PTE'

A következő függvénnyel tudunk kérdéseket feltenni a megtanult doksikkal kapcsolatban.

In [75]:
def inputFunction(question):
  k = 3 # How many documents you want to retrieve, defaults to 10, we set it to 3 here for readability
  results = RAG.search(query=question, k=k)
  return results[0]['content'].replace("\n","").replace("   ","  ").replace("  "," ")

Tesztként:

In [76]:
print(inputFunction("Mikor utasítja el érdemi vizsgálat nékül az EHÖK a kollégiumi jelentkezésemet?"))

Loading searcher for index PTE for the first time... This may take a few seconds
[Apr 06, 15:30:55] #> Loading codec...
[Apr 06, 15:30:55] #> Loading IVF...
[Apr 06, 15:30:55] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 746.85it/s]

[Apr 06, 15:30:55] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 11.12it/s]

Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . Mikor utasítja el érdemi vizsgálat nékül az EHÖK a kollégiumi jelentkezésemet?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2771, 21815, 28981, 28032,  3900,  3449,  9413,  3207,
         4328, 26619, 28745,  7911,  2102, 11265,  5283,  2140, 17207, 15501,
         6559,  1037, 12849,  6216,  5856, 12717, 15333, 16136,  3489, 11254,
        21382,   102], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')

A hibásan kitöltött pályázati űrlapot a Küldöttgyűlés érdemi vizsgálat nélkül elutasítja. A hallgató a PTE EHÖK Küldöttgyűlésének döntése ellen, jogszabály vagy szabályzatsértésre hivatkozássa l fellebbezéssel élhet, melyet a Másodfokú Tanulmányi Bizottságnak címezve a PTE EHÖK Küldöttgyűléséhez kell benyújtani a tudomásszerz

A Gradio-t is fel kell telepítenünk a webes GUI miatt.

In [78]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.4/313.4 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 100.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.9 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=7600888066e040c485ef3c4e9c6265f2886ac71e7d58363100d4ccb85fc48805
  Stored in directory: /root/.cache/pip/wheels/bd/65/9a/671fc6dcde07d4418df0c592f8df512b26d7a0029c2a23dd81
Successfully built ffmpy


Megvalósítunk egy egyszerű webes GUI-t:

In [83]:
import gradio as gr

# Létrehozunk egy Gradio interfészt a függvényünkkel
interface = gr.Interface(
    fn=inputFunction,
    inputs="text",
    outputs="text",
    title="RAG Model Summary Generator (PTE admin edition)",
    description="Generates a summary for a given input text using the RAG model. This RAG model trained on many administrative documents of PTE and PTE EHÖK.",
)

# A Gradio interfész indítása
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1188b672b868b5fe61.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
